In [1]:
import polars as pl

In [80]:
userDF = pl.read_json('users-extracted.json')
userEventDF = pl.read_json('user_events-extracted.json')
userTransDF = pl.read_json('user_transactions-extracted.json')
fbDF = pl.read_json('facebook_ads-extracted.json')
igDF = pl.read_json('instagram_ads-extracted.json')

In [81]:
ctx = pl.SQLContext().register_many({'users':userDF, 'user_events':userEventDF, 'user_transactions':userTransDF, 'fb':fbDF, 'ig':igDF})

In [89]:
dim_user = ctx.execute('''
SELECT
id AS user_id,
client_id,
first_name || ' ' || last_name AS full_name,
email,
dob,
gender,
register_date,
CASE
    WHEN fb.id IS NOT NULL THEN 'Facebook'
    WHEN ig.id IS NOT NULL THEN 'Instagram'
    ELSE 'Unknown'
END AS ads_source,
FROM users u
LEFT JOIN fb on fb.id = u.client_id
LEFT JOIN ig on ig.id = u.client_id
''', eager=True)

dim_user.write_json('dim_user.json', row_oriented=True)

In [94]:
dim_ads = ctx.execute('''
SELECT
'Facebook' AS ads_source, id, ads_id, device_type, device_id, timestamp
FROM fb
UNION ALL
SELECT
'Instagram' AS ads_source, id, ads_id, device_type, device_id, timestamp
FROM ig
''', eager=True)
dim_ads.write_json('dim_ads.json', row_oriented=True)

In [162]:
fact_user_performance = ctx.execute('''
SELECT 
    id AS user_id,
    MAX(user_events.timestamp) AS last_activity,
    COUNT(DISTINCT user_events.id) AS total_events,
    COUNT(DISTINCT CASE WHEN user_events.event_type = 'login' THEN user_events.id ELSE NULL END) AS total_logins,
    COUNT(DISTINCT CASE WHEN user_events.event_type = 'search' THEN user_events.id ELSE NULL END) AS total_searches,
    COUNT(DISTINCT CASE WHEN user_events.event_type = 'purchase' THEN user_events.id ELSE NULL END) AS total_purchases,
    SUM(CASE WHEN user_transactions.transaction_type = 'purchase' THEN user_transactions.amount ELSE 0 END) AS total_purchase_amount
    FROM users u
LEFT JOIN user_events on user_events.user_id = u.id
LEFT JOIN user_transactions on user_transactions.user_id = u.id
WHERE user_events.timestamp IS NOT NULL
GROUP BY id
''', eager=True)
fact_user_performance.write_json('fact_user_performance.json', row_oriented=True)

In [155]:
fact_ads_performance = ctx.execute('''
SELECT
    a.ads_id,
    COUNT(DISTINCT a.id) AS total_clicks,
    COUNT(DISTINCT CASE WHEN a.ads_source = 'Facebook' THEN a.id END) AS total_facebook_clicks,
    COUNT(DISTINCT CASE WHEN a.ads_source = 'Instagram' THEN a.id END) AS total_instagram_clicks,
    COUNT(DISTINCT CASE WHEN u.id IS NOT NULL THEN a.id END) AS total_converted,
    SUM(CASE WHEN t.transaction_type = 'purchase' THEN 1 ELSE 0 END) AS total_purchases,
    SUM(CASE WHEN t.transaction_type = 'purchase' THEN t.amount ELSE 0 END) AS total_purchase_amount
FROM (
    SELECT id, ads_id, 'Facebook' AS ads_source FROM fb
    UNION ALL
    SELECT id, ads_id, 'Instagram' AS ads_source FROM ig
) AS a
LEFT JOIN dibimbing.user.users AS u ON a.id = u.client_id
LEFT JOIN dibimbing.event."User Event" AS e ON a.ads_id = e.event_data->>'ads_id'
LEFT JOIN dibimbing.user.user_transactions AS t ON u.id = t.user_id AND t.transaction_type = 'purchase'
GROUP BY a.ads_id
''', eager=True)

ComputeError: sql parser error: Expected end of statement, found: ON at Line: 6, Column 17